In [1]:
from glob import glob
import IPython.display as ipd
import matplotlib.pyplot as plt
from pyAudioAnalysis import audioBasicIO as aIo
from pyAudioAnalysis import ShortTermFeatures as sf
import librosa
import librosa.display
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
file_paths = glob('../Downloads/Raw/Video/Segmented/*')

In [3]:
data = pd.read_csv('../Downloads/Raw/OpinionLevelSentiment.csv', names = ['Seg Start', 'Segment end', 'Name','Seg No', 'Review'])

In [4]:
dataframe = pd.DataFrame(data)
dataframe

,Seg Start,Segment end,Name,Seg No,Review
0,27.879138,32.927664,_dI--eQ6qVU,1,2.4
1,45.548980,47.783900,_dI--eQ6qVU,2,3.0
2,47.783900,59.247846,_dI--eQ6qVU,3,-0.2
3,59.247846,68.277324,_dI--eQ6qVU,4,3.0
4,68.277324,71.200680,_dI--eQ6qVU,5,3.0
...,...,...,...,...,...
2194,147.217914,153.703175,ZUXBRvtny7o,30,0.4
2195,157.265079,161.256009,ZUXBRvtny7o,31,1.4
2196,161.256009,173.049206,ZUXBRvtny7o,32,0.4
2197,173.049206,175.224263,ZUXBRvtny7o,33,0.8


In [5]:
dataframe['Polarity'] = dataframe['Review']
for i in range(len(dataframe)):
    if(dataframe['Review'][i]>0):
        #positive
        dataframe['Polarity'][i] = 1
    else:
        #negative
        dataframe['Polarity'][i] = 0

In [6]:
dataframe

,Seg Start,Segment end,Name,Seg No,Review,Polarity
0,27.879138,32.927664,_dI--eQ6qVU,1,2.4,1.0
1,45.548980,47.783900,_dI--eQ6qVU,2,3.0,1.0
2,47.783900,59.247846,_dI--eQ6qVU,3,-0.2,0.0
3,59.247846,68.277324,_dI--eQ6qVU,4,3.0,1.0
4,68.277324,71.200680,_dI--eQ6qVU,5,3.0,1.0
...,...,...,...,...,...,...
2194,147.217914,153.703175,ZUXBRvtny7o,30,0.4,1.0
2195,157.265079,161.256009,ZUXBRvtny7o,31,1.4,1.0
2196,161.256009,173.049206,ZUXBRvtny7o,32,0.4,1.0
2197,173.049206,175.224263,ZUXBRvtny7o,33,0.8,1.0


In [142]:
p = 0
n = 0
for i in range(2199):
    if(dataframe['Polarity'][i] == 1):
        p+=1
    else:
        n+=1

In [143]:
p

1080

In [144]:
n

1119

In [7]:
def load_file(filepath):
    y,sr = librosa.load(filepath)
    return (y,sr)

In [8]:
def divide_segments(y_time,start):
    # divide into 50 ms segments
    # 50ms -> means 1102 samples    
    y_segmented = y_time[start:start+1102]
    return y_segmented

In [9]:
def stft(y_seg , sr, window, step):
    [f,f_name] = sf.feature_extraction(y_seg,sr, window, step,  deltas = False)
    return f

In [169]:
def feature_matricization(y_time):
    y_seg = divide_segments(y_time,0)
    y_stft = stft(y_seg,sr,0.05*sr,0.05*sr)
    y_stft = y_stft.reshape(34)
    
    start = 1111
    y_stfts = y_stft
    while(start <= len(y_time)-1111):
        y_seg = divide_segments(y_time,start)
        #print(y_seg)
        start += 1111
        y_stft = stft(y_seg,sr,0.05*sr,0.05*sr)
        #y_stft_arr = np.asarray(y_stft).T
        y_stft = y_stft.reshape(34)
        y_stfts = np.vstack((y_stfts, y_stft))
        #print(y_stft_arr.shape)

        #np.column_stack((y_stfts, y_stft_arr))
        
    if(len(y_stfts)<140):
        #cal how less
        x = 140 - len(y_stfts)%140
        zeros_arr = np.zeros(shape = (x,34))
        x_arr = np.asanyarray(y_stfts)
        x_arr = x_arr.reshape(-1,34)
        x_final = np.vstack((x_arr,zeros_arr))
        
        return x_final
    if(len(y_stfts)>=140):
        y_stfts = y_stfts[0:140]
        return np.asanyarray(y_stfts)

In [170]:
file_path = '../Downloads/Raw/Video/Segmented/' + dataframe['Name'][0] +  '_' + dataframe['Seg No'][0].__str__() +'.mp4'
label = dataframe['Polarity'][0]
labels = np.asarray(label)
y,sr = librosa.load(file_path)
y_time = librosa.frames_to_time(y)
feat = feature_matricization(y_time)
features = feat
for i in range(len(file_paths)):
    if(i!=0):
        #print(i)
        file_path = '../Downloads/Raw/Video/Segmented/' + dataframe['Name'][i] +  '_' + dataframe['Seg No'][i].__str__() +'.mp4'
        label = dataframe['Polarity'][i]
        labels = np.vstack((labels,label))
        y,sr = librosa.load(file_path)
        y_time = librosa.frames_to_time(y)
        feat = feature_matricization(y_time)
        features = np.vstack((features,feat))

In [171]:
labels.shape

(2199, 1)

In [172]:
features.shape

(307860, 34)

In [173]:
# divide features into 300 * 34 segements

In [174]:
# no of segments
indices = features.shape[0]/140
indices

2199.0

In [175]:
features_segments = np.split(features,indices)

In [224]:
feat_seg_arr = np.asanyarray(features_segments)

In [225]:
feat_seg_arr.shape

(2199, 140, 34)

In [226]:
labels = labels.astype('float32')

In [227]:
feat_seg_arr

array([[[9.08265213e-02, 1.43189122e-01, 3.20268863e+00, ...,
         3.09469503e-02, 3.52974396e-03, 1.80760169e-02],
        [7.90190736e-02, 2.02055657e-02, 1.75422621e+00, ...,
         3.17594920e-02, 7.93433810e-03, 1.30776348e-02],
        [9.44595822e-02, 1.22987892e-01, 3.10176730e+00, ...,
         1.48196364e-03, 4.75668870e-04, 8.72761968e-03],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[1.08991826e-01, 7.23311348e-02, 3.20115993e+00, ...,
         6.35653628e-03, 3.20171556e-03, 1.19085077e-02],
        [1.03542234e-01, 9.25700530e-02, 2.95331418e+00, ...,
         4.01461759e-02, 4.96482958e-03, 1.60900867e-02],
        [1.72570391e-01, 

In [215]:
feat_seg_arr[1][1]

array([ 1.03542234e-01,  9.25700530e-02,  2.95331418e+00,  3.49500837e-01,
        2.96975051e-01,  1.78752607e+00,  0.00000000e+00,  5.13611615e-01,
       -2.06350097e+01,  1.74653738e+00,  5.54112699e-01,  1.37389003e-01,
        2.40863129e-01,  8.00377026e-02,  1.05571110e-01,  3.32976985e-01,
        3.33061559e-01, -9.30991838e-02, -2.07713672e-01, -8.39511556e-02,
       -1.69771896e-01,  2.40287606e-02,  8.64434975e-03,  5.03347315e-02,
        1.03075816e-02,  6.07012424e-03,  2.98686576e-03,  3.76430478e-02,
        1.54078890e-03,  5.74301145e-03,  1.04175462e-02,  4.01461759e-02,
        4.96482958e-03,  1.60900867e-02])

In [230]:
feat_seg_arr_norm = feat_seg_arr
for i in range(feat_seg_arr.shape[0]):
    for j in range(feat_seg_arr.shape[1]):
        for k in range(feat_seg_arr.shape[2]):
            if(feat_seg_arr[i][j].max()!=0):
                
                feat_seg_arr_norm[i][j][k] = feat_seg_arr[i][j][k]/feat_seg_arr.max()

KeyboardInterrupt: 

In [231]:
feat_seg_arr_norm

array([[[1.53109488e-02, 2.41378981e-02, 5.39888578e-01, ...,
         5.21683715e-03, 5.95021456e-04, 3.04713827e-03],
        [1.33205254e-02, 3.40612385e-03, 2.95716132e-01, ...,
         5.35381018e-03, 1.33751951e-03, 2.20454326e-03],
        [1.59233867e-02, 2.07325051e-02, 5.22875912e-01, ...,
         2.49819866e-04, 8.01851881e-05, 1.47124580e-03],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[1.83731385e-02, 1.21931159e-02, 5.39630880e-01, ...,
         1.07154386e-03, 5.39724544e-04, 2.00745937e-03],
        [1.74544816e-02, 1.56048621e-02, 4.97850643e-01, ...,
         6.76758322e-03, 8.36938928e-04, 2.71236297e-03],
        [2.90908026e-02, 

In [232]:
# train test split

In [233]:
from sklearn.model_selection import train_test_split

In [234]:
X_train, X_test, y_train, y_test = train_test_split(feat_seg_arr_norm, labels)

In [235]:
print(X_train[10] == X_train[1])

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]


In [251]:
X_train.shape

(1649, 140, 34)

In [260]:
y_train.shape

(1649, 1)

In [236]:
import tensorflow as tf

In [272]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout

In [263]:
model = Sequential()

In [288]:
model = Sequential()
model.add(layers.Conv1D(256, 5,padding='same',
                 input_shape=(140,34)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling1D(pool_size=(8)))
model.add(layers.Dropout(0.2))
model.add(layers.Conv1D(128, 5,padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling1D(pool_size=(4)))
model.add(layers.Dropout(0.1))
model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

In [289]:
model.compile(optimizer = 'Adam' , loss = tf.keras.losses.binary_crossentropy, metrics = ['accuracy'])

In [290]:
model.summary()

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_86 (Conv1D)          (None, 140, 256)          43776     
                                                                 
 activation_24 (Activation)  (None, 140, 256)          0         
                                                                 
 max_pooling1d_84 (MaxPoolin  (None, 17, 256)          0         
 g1D)                                                            
                                                                 
 dropout_25 (Dropout)        (None, 17, 256)           0         
                                                                 
 conv1d_87 (Conv1D)          (None, 17, 128)           163968    
                                                                 
 activation_25 (Activation)  (None, 17, 128)           0         
                                                     

In [291]:
model.fit(X_train,y_train,epochs=20)

Epoch 1/20
52/52 [==============================] - 3s 32ms/step - loss: 1.3326 - accuracy: 0.4967
Epoch 2/20
52/52 [==============================] - 2s 30ms/step - loss: 0.6926 - accuracy: 0.5264
Epoch 3/20
52/52 [==============================] - 2s 37ms/step - loss: 0.6848 - accuracy: 0.5585
Epoch 4/20
52/52 [==============================] - 2s 32ms/step - loss: 0.6884 - accuracy: 0.5561
Epoch 5/20
52/52 [==============================] - 2s 31ms/step - loss: 0.6802 - accuracy: 0.5597
Epoch 6/20
52/52 [==============================] - 2s 33ms/step - loss: 0.6812 - accuracy: 0.5822
Epoch 7/20
52/52 [==============================] - 2s 30ms/step - loss: 0.6697 - accuracy: 0.5810
Epoch 8/20
52/52 [==============================] - 2s 32ms/step - loss: 0.6771 - accuracy: 0.5767
Epoch 9/20
52/52 [==============================] - 2s 30ms/step - loss: 0.6692 - accuracy: 0.5943
Epoch 10/20
52/52 [==============================] - 2s 30ms/step - loss: 0.6675 - accuracy: 0.5858
Epoch 11/

In [292]:
from sklearn.metrics import accuracy_score

In [365]:
p = model.predict(X_test)

In [366]:
for i in range(len(p)):
    if p[i]>0.65:
        p[i] = 1.0
    else:
        p[i] = 0.0
p

array([[0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],

In [367]:
y_test

array([[0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],

In [368]:
p_train = model.predict(X_train)

In [369]:
p_train

array([[0.2146877 ],
       [0.6415029 ],
       [0.42251328],
       ...,
       [0.49033993],
       [0.5063693 ],
       [0.38211235]], dtype=float32)

In [370]:
y_train

array([[0.],
       [1.],
       [0.],
       ...,
       [0.],
       [1.],
       [1.]], dtype=float32)

In [371]:
from sklearn.metrics import accuracy_score

In [372]:
accuracy = accuracy_score(y_test,p)

In [373]:
accuracy

0.5509090909090909